# Optimizing SGP4

Kerry N. Wood

kerry.n.wood@gmail.com

October 13, 2022


In [1]:
import numpy as np
from datetime import datetime, timedelta
import multiprocessing
import itertools
import time
import astropy.time
import numpy as np
import itertools
import scipy.interpolate
import pandas as pd

from sgp4.io import twoline2rv
from sgp4.earth_gravity import wgs72
from sgp4.propagation import sgp4 as sgprop
from sgp4.conveniences import jday_datetime
from sgp4.ext import invjday

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
def read_twoline_debug( fn ):
    '''
    reads in a file like:
    1 04523U 69082 GA 93237.23759955  .06028875 +36000-5 +45037-2 0  9999
    2 04523 070.0374 356.6819 0011002 283.3732 077.1114 16.15010855249410
    1 04524U 70071  A 70263.60375341  .00109633 +00000-0 +00000-0 0  9999
    2 04524 072.8586 118.4837 0110530 020.4754 340.0801 16.00284599001952
    1 04528U 70072  B 70256.72128638  .02030310 +00000-0 +00000-0 0  9991
    2 04528 051.5351 195.5582 0026658 320.0288 039.8725 16.26016213000198
    ....
    '''
    with open(fn) as F: lines = F.readlines()
    L1 = list( filter( lambda X: X[0] == '1', lines ) )
    L2 = list( filter( lambda X: X[0] == '2', lines ) )
    assert len(L1) == len(L2)
    tles = [ twoline2rv(A,B,wgs72) for A,B in zip(L1,L2) ]
    return zip( L1, L2, tles )

tles = list(  read_twoline_debug('../catalog22.twoline') )

In [9]:
%%prun -s tottime
xpdotp   =  1440.0 / (2.0 * np.pi)

def propagateRun( tleobj, step=1 ):
    # find the revs per day from the TLE, so we can figure out period
    revs_per_day = tleobj.no_kozai * xpdotp 
    # go about 2 days
    prop_interval = 7 * (1440 / revs_per_day)
    offsets = np.arange( -step*5, prop_interval, step+5 )
    def propToJD( offset ): return np.hstack( sgprop( tleobj, offset ) )
    eph = np.vstack( [propToJD(O) for O in offsets])
    return eph

data   = pd.DataFrame( tles , columns=['l1','l2','tle'])
testset = data.sample( 1000, replace=False )
testset['tle'].apply( propagateRun )